In [ ]:
# 1. Uninstall numpy versi 2.x yang bikin kacau
!pip uninstall -y numpy

# 2. Install numpy versi 1.26.4 (Versi paling stabil untuk PyTorch saat ini)
!pip install "numpy==1.26.4"

print("\n NOW RESTART SESSION!")
print("If not restarted, Python will still use the old Numpy in memory..")

In [ ]:
# 1. Install Ultralytics (YOLOv8)
!pip install -q ultralytics

# 2. Import & Cek GPU
import ultralytics
from ultralytics import YOLO
import os
import shutil
import yaml

# Clean working directory
if os.path.exists("/kaggle/working/coco"):
    shutil.rmtree("/kaggle/working/coco")
    
ultralytics.checks()
print("\n Setup complete. GPU Ready.")

In [ ]:
import os
import shutil

# Path working directory Kaggle
folder = '/kaggle/working'

print(f"Cleaning {folder}...")

for filename in os.listdir(folder):
    
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path) # Hapus file/symlink
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path) # Hapus folder beserta isinya
    except Exception as e:
        print(f'❌ Failed cleaning {file_path}. Reason: {e}')

print("Done! The /kaggle/working folder is now completely empty.")

In [ ]:
import os
import shutil
import json
import glob
import yaml
import numpy as np
import torch
from PIL import Image
from tqdm.notebook import tqdm
from ultralytics import YOLO

# Cek Versi Numpy (Harus 1.26.x)
print(f"✅ Numpy Version: {np.__version__}")
print(f"✅ Torch Version: {torch.__version__}")

# ================= 1. SETUP ULANG DATASET (Karena Session Restart) =================
INPUT_ROOT = "/kaggle/input/palembang-telecomunication-provider-infrastructure"
JSON_PATH = os.path.join(INPUT_ROOT, "coco/annotations/instances_annotations_cvat.json")
OUTPUT_DIR = "/kaggle/working/yolo_dataset"

if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
for split in ['train', 'val']:
    os.makedirs(os.path.join(OUTPUT_DIR, "images", split), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "labels", split), exist_ok=True)

# Load JSON & Copy Images
with open(JSON_PATH, 'r') as f: coco = json.load(f)
categories = {cat['id']: cat['name'] for cat in coco['categories']}
id_map = {old_id: new_id for new_id, old_id in enumerate(sorted(categories.keys()))}
names_list = [categories[old_id] for old_id in sorted(categories.keys())]

# Map Source Images
source_map = {}
for path in glob.glob(f"{INPUT_ROOT}/**/*.*", recursive=True):
    if path.lower().endswith(('jpg', 'jpeg', 'png')):
        source_map[os.path.basename(path)] = path

# Process Images
images = coco['images']
split_idx = int(len(images) * 0.8)
splits = {'train': images[:split_idx], 'val': images[split_idx:]}

print("Reshaping Dataset...")
for split, img_list in splits.items():
    for img_info in tqdm(img_list, desc=f"Processing {split}"):
        basename = os.path.basename(img_info['file_name'])
        if basename not in source_map: continue
        
        # Save Image (PIL untuk keamanan)
        with Image.open(source_map[basename]) as im:
            im = im.convert('RGB')
            im.save(os.path.join(OUTPUT_DIR, "images", split, os.path.splitext(basename)[0] + ".jpg"), quality=95)
            w_real, h_real = im.size

        # Save Label
        txt_path = os.path.join(OUTPUT_DIR, "labels", split, os.path.splitext(basename)[0] + ".txt")
        with open(txt_path, 'w') as f_txt:
            anns = [a for a in coco['annotations'] if a['image_id'] == img_info['id']]
            for ann in anns:
                bbox = ann['bbox']
                xc, yc = (bbox[0] + bbox[2]/2)/w_real, (bbox[1] + bbox[3]/2)/h_real
                w, h = bbox[2]/w_real, bbox[3]/h_real
                f_txt.write(f"{id_map[ann['category_id']]} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")

# Config YAML
yaml_content = {'path': OUTPUT_DIR, 'train': 'images/train', 'val': 'images/val', 'names': {i: n for i, n in enumerate(names_list)}}
with open(f"{OUTPUT_DIR}/data.yaml", 'w') as f: yaml.dump(yaml_content, f)

In [ ]:
# ================= 2. TRAINING (NORMAL MODE) =================
print("\n🚀 Train Starting (Stable Environment)...")
model = YOLO("yolov8s.pt")

# Kita tetap matikan workers (0) untuk keamanan maksimal di Kaggle
# Tapi kita nyalakan kembali augmentasi karena Numpy sudah benar
model.train(
    data=f"{OUTPUT_DIR}/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device=0,
    workers=0,      # Tetap 0 agar loading stabil
    project="provider_project",
    name="final_run",
    exist_ok=True
)

In [ ]:
# Validasi ulang dengan best.pt
working_dir = "/kaggle/working/yolo_dataset"

best_model_path = "/kaggle/input/yolov8s-telecom-infrastructure-detection/pytorch/default/1/provider_yolov8s.pt"
model_best = YOLO(best_model_path)

print("\nBest Model Evaluation:")
metrics = model_best.val(
    data=f"{working_dir}/data.yaml",
    augment=True   # <--- Tambahkan ini. Magic happens here.
)

print(f"\n mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

In [ ]:
import glob
import random

working_dir = "/kaggle/working/yolo_dataset"
# Ambil gambar acak dari dataset validasi untuk tes
test_images = glob.glob(f"{working_dir}/images/val/*")
# Pilih 3 gambar acak
sample_images = random.sample(test_images, 3) if len(test_images) >= 3 else test_images

print("Making predictions on samples...")

results = model_best.predict(
    source=sample_images,
    conf=0.4,       # Confidence threshold (di bawah 40% dianggap noise)
    save=True,      # Simpan gambar hasil
    project="provider-marker",
    name="inference_test",
    exist_ok=True
)

# Tampilkan hasil (Khusus Notebook)
from IPython.display import Image, display

print("\nDetection Result:")
for img_path in glob.glob("/kaggle/working/provider-marker/inference_test/*.jpg"):
    display(Image(filename=img_path, width=600))

In [ ]:
from ultralytics import YOLO
import glob
import random
from IPython.display import Image, display

# --- BAGIAN KRUSIAL YANG HILANG ---
# Inisialisasi model dengan path weights yang kamu punya
weights_path = "/kaggle/input/yolov8s-telecom-infrastructure-detection/pytorch/default/1/provider_yolov8s.pt"

print(f"Loading the model 'brain' from: {weights_path}")
# Load modelnya. Jangan lupa task='detect' implisit di YOLOv8 biasanya
model = YOLO(weights_path) 
# ----------------------------------

working_dir = "/kaggle/working/yolo_dataset"

# Validasi jalur dulu, jangan asal gas. Skeptis itu perlu.
test_images = glob.glob(f"{working_dir}/images/val/*.jpg") # Tambahkan ekstensi file biar spesifik

if not test_images:
    print("It's completely empty! Check your 'working_dir' path. Is there anything in the images/val folder?")
else:
    # Pilih 3 gambar acak
    sample_images = random.sample(test_images, 3) if len(test_images) >= 3 else test_images

    print(f"🕵️ Melakukan prediksi pada {len(sample_images)} sampel...")

    # Ganti 'model_best' jadi 'model' sesuai definisi di atas
    results = model.predict(
        source=sample_images,
        conf=0.4,       # Threshold 40%. Kalau noise banyak, naikkan ke 0.5 atau 0.6.
        save=True,      
        project="provider-marker",
        name="inference_test",
        exist_ok=True,
    )

    print("\nDetection Result:")
    # Ambil hasil dari folder project/name yang baru dibuat
    # Hati-hati, YOLO kadang membuat folder 'predict', 'predict2', dst jika exist_ok=False.
    # Tapi karena exist_ok=True, dia akan menimpa atau memakai folder yang sama.
    for img_path in glob.glob("/kaggle/working/provider-marker/inference_test/*.jpg"):
        display(Image(filename=img_path, width=600))

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO

# ================= 1. CONFIG =================
# Path model (Gunakan hasil training terbaikmu)
# Sesuaikan 'yolo_m_800_augmented' dengan nama project yang kamu pakai terakhir
model_path = "/kaggle/input/yolov8s-telecom-infrastructure-detection/pytorch/default/1/provider_yolov8s.pt" 
# Jika error file not found, ganti ke: "/kaggle/working/provider_project/final_run/weights/best.pt"

# --- 📝 ISI PATH GAMBAR DI SINI ---
target_images = [
    "/kaggle/input/palembang-telecomunication-provider-infrastructure/test/test_0001.jpeg"
]
# =============================================

# 2. Load Model
if not os.path.exists(model_path):
    print(f"Model not found in: {model_path}")
    print("Try checking the output folder on the right for the correct path..")
else:
    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)

    # 3. Loop Inference
    for img_path in target_images:
        print(f"\nProcessing: {os.path.basename(img_path)}...")
        
        if not os.path.exists(img_path):
            print(f"File not found: {img_path}")
            continue

        # Lakukan Prediksi
        results = model.predict(
            source=img_path,
            conf=0.25,        # Ambang batas keyakinan (25%)
            iou=0.45,         # NMS Threshold (biar kotak gak tumpuk-tumpuk)
            save=False,       # Gak usah save file, kita plot langsung
            verbose=False,    # Supaya log gak berisik
            augment=True
        )
        
        # --- TAMPILKAN HASIL ---
        result = results[0]
        
        # A. Hitung Objek (Text Report)
        class_counts = {}
        for box in result.boxes:
            cls_id = int(box.cls[0])
            cls_name = model.names[cls_id]
            conf = float(box.conf[0])
            
            # Hitung jumlah per kelas
            class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
            print(f"   ➤ Detected: {cls_name} ({conf*100:.1f}%)")
            
        if not class_counts:
            print("   (No objects detected)")
        else:
            print(f"   Summary: {class_counts}")

        # B. Tampilkan Gambar (Visual Report)
        # Plot hasil (mengembalikan array numpy BGR)
        res_plotted = result.plot()
        
        res_rgb = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(res_rgb)
        plt.title(f"Hasil: {os.path.basename(img_path)}", fontsize=12)
        plt.axis('off')
        plt.show()